# Pokračování v prozkoumávání Pythonu
V této lekci se budeme zabývat pokročilejšími funkcemi Pythonu. Budeme se zabývat:

- prostory jmen a obory platnosti proměnných,
- Jupyter magics,
- třídami
- výjimkami,
- práci s řetězci,
- generátory a iterátory,
- list comprehension,
- práci se soubory,

# Prostory jmen a obor platnosti proměnných
Prostor jmen (anglicky `namespace`) je struktura sloužící k organizaci názvů proměnných, funkcí, tříd a dalších objektů. 
Dá se představit jako slovník, kde klíčem je název objektu (název proměnné) a hodnotou je objekt sám.

V Pythonu existuje čtyři druhy prostorů jmen:
- vestavěný (`builtins`)
    - jedná se o prostor jmen, který je vytvořen při spuštění Python interpretu
- globální (`global`)
    - jedná se o prostor jmen, který je vytvořen při vstupu do interaktivního režimu nebo při spuštění skriptu
- ne-lokální (`nonlocal`)
    - jedná se o prostor jmen, který je v bloku do nějž je zanořen aktuální blok
- lokální (`local`)
    - jedná se o prostor jmen, který je vytvořen při vstupu do odděleného bloku = nejčastěji funkce

Toto pořadí (od posledního k prvnímu) je také pořadí, v jakém se prohledávají prostory jmen. Pokud se název objektu nenajde v lokálním prostoru jmen, tak se prohledává ne-lokální prostor jmen, pak globální a nakonec vestavěný. Pokud se objekt v žádném prostoru jmen nenajde, tak se vyvolá výjimka `NameError`.

Globální a lokální prostor jmen jsou skutečně implementovány jako slovníky. Vestavěný prostor jmen je implementován jako modul `builtins`. 

In [ ]:
nejaka_promenna = 1
print(type(globals()))
print(globals())


In [ ]:
nejaka_promenna = 1

def funkce():
    nova_promenna_uvnitr_funkce = 1
    nejaka_promenna = 2
    print(type(locals()))
    print(locals())
    
funkce()

In [ ]:
print(type(__builtins__))
print(dir(__builtins__))

In [ ]:
# pokud proměnná není v žádném jmeném prostoru, vyhodíme chybu
print(nedefinovana_promenna)
nedefinovana_promenna = 1

In [ ]:
# pokud vyrobíme novou proměnnou zde, bude v globálním prostoru
moje_promenna = 42
print(id(moje_promenna))
print(id(globals()['moje_promenna']))


In [ ]:
# ukázka globálního prostoru
moje_promenna = 42


def funkce():
    print(moje_promenna)


funkce()


In [ ]:
# ukázka lokálního prostoru
moje_promenna = 42


def funkce():
    moje_promenna = 43
    print(moje_promenna)


funkce()


In [ ]:
# ukázka nelokálního prostoru
moje_promenna = 42


def funkce():
    moje_promenna = 43

    def funkce2():
        print(moje_promenna)

    funkce2()


funkce()


Pro lokální prostor jmen je vytvořen nový slovník ve chvíli, kdy je funkce definována. Tedy ne při běhu!

In [ ]:
# výstup tohote není 42, 43! ale chyba
moje_promenna = 42


def funkce():
    print(moje_promenna)
    moje_promenna = 43
    print(moje_promenna)


funkce()
# všimněmě si, že se nejedná o NameError, ale o UnboundLocalError

Přístup k objektům z globálního prostoru jmen, umožňuje objekt měnit, ale nemůže jej přepsat. Pokud se pokusíme přepsat objekt, tak se přesuneme do lokálního prostoru jmen.

In [ ]:
muj_list = [1, 2, 3]


def funkce():
    print(muj_list)
    muj_list[0] = 42
    muj_list.append(4)


funkce()
print(muj_list)


Pokud jej chceme přepsat, tak musíme použít klíčové slovo `global`. Asi nemusíme zdůrazňovat, že je to poněkud nerozumné řešení z pohledu transparentnosti kódu.

In [ ]:
muj_list = [1, 2, 3]

print(id(muj_list))


def funkce():
    global muj_list
    muj_list = [42, 2, 3]


funkce()

print(id(muj_list))
print(muj_list)


Úplně stejně funguje i přístup k objektům z ne-lokálního prostoru jmen. Pokud jej chceme přepsat, tak musíme použít klíčové slovo `nonlocal`.

In [ ]:
# ukázka nonlocal
def funkce():
    muj_list = [1, 2, 3]

    def funkce2():
        nonlocal muj_list
        muj_list[0] = 42
        muj_list.append(4)

    funkce2()
    print(muj_list)


funkce()


Asi už je teď trochu jasné i jak funguje obor platnosti proměnných. Raději ale poslední ukázka:

In [ ]:
prommena = 42

def funkce():
    prommena = 43
    print("1: ", prommena)
    def funkce2():
        prommena = 44
        print("2: ", prommena)
        
    funkce2()
    print("3: ", prommena)
    
print("4: ", prommena)
funkce()
print("5: ",prommena)

Vzpomínáte si, že jsme říkali, že lokální prostor je vytvořen přímo při definici funkce? Podobně to funguje pro seznam defaultních hodnot parametrů. Ten se po deklaraci funkce vytvoří a zůstává stejný pro všechny volání této funkce. Pokud je defaultní hodnota parametru měnitelného typu, tak se tyto změny projeví i v dalších voláních této funkce. 

Pozor, i když objekt uvnitř funkce přepíšeme, tak originální defaultní hodnota je stále zachována a připravena pro další volání funkce.

In [ ]:
def funkce(muj_list = []):
    muj_list.append(42)
    print(muj_list)
    muj_list = []
    
funkce()
funkce()
funkce([])
funkce()
funkce()


Na defaultní hodnoty parametrů funkce se lze podívat pomocí atributu `__defaults__`.

In [ ]:
def funkce(muj_list=[], muj_string=""):
    muj_list.append(42)
    muj_string += "42"
    print(muj_list)
    print(muj_string)


print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)

# Jupyter magics a další
Jupyter magics jsou speciální příkazy spustitelné v Jupyter noebooku, které umožňují využívat různé funkce mimo prostředí Pythonu. 

Jupyter magics se spouští pomocí prefixu `%` (pro jednořádkové magics) nebo `%%` (pro víceřádkové magics).

Příkazů je obrovské množství, viz například [tento seznam](https://ipython.readthedocs.io/en/stable/interactive/magics.html). Zde si uvedeme pouze pár.

Nápovědu (docstring) lze k magics získat pomocí `?` za příkazem.

**Jednořádkové magics:**
- `%time` - zobrazí čas, který trvalo vykonat příkaz
- `%timeit` - pokročilejší verze příkazu `%time`, který spustí příkaz vícekrát a zobrazí statistiku
- `%magic` - zobrazí nápovědu k magics (výpis všech dostupných magics)
- `%lsmagic` - zobrazí seznam dostupných magics
- `%load` - načte zdrojový kód z externího souboru do aktuální buňky
- `%run` - spustí externí skript
- `%less` / `%more` / `%cat` / `%pycat` - zobrazí externí soubor v textovém editoru
- `%system` / `!!` - spustí příkaz v systémovém terminálu - s vazbou na Jupyter (chová se jinak než `!`)
- `%pwd` / `%cd` - zobrazí / změní aktuální pracovní adresář
- `%who` / `%whos` - zobrazí seznam proměnných
- `%ls` / `%ll` - zobrazí obsah aktuálního adresáře


**Víceřádkové magics:**
Týkají se celé buňky, ne jen jednoho příkazu.
- `%%time` - zobrazí čas, který trvalo vykonat obsah buňky
- `%%timeit` - pokročilejší verze příkazu `%%time`, který spustí buňku vícekrát a zobrazí statistiku
- `%%writefile` / `%%file` - zapsat obsah buňky do externího souboru
- `%%bash` / `%%html` / `%%javascript` / `%%latex` / ... - spustí buňku s kódem v daném jazyce
- `%%prun` - spustí buňku pomocí profilovacího modulu `cProfile`


In [ ]:
%magic

In [ ]:
seznam_magickych_funkci = %lsmagic
print(type(seznam_magickych_funkci))
print(seznam_magickych_funkci)


In [ ]:
!mkdir test

In [ ]:
!!ls

In [ ]:
%cd test

In [ ]:
%pwd

In [ ]:
%%writefile?

In [ ]:
%%writefile test.py
def funkce():
    a = 2
    print("Ahoj")
funkce()

In [ ]:
%ll

In [ ]:
%less test.py

In [ ]:
%run test.py

In [ ]:
%who

In [ ]:
%time funkce()

In [ ]:
def najdi_prvocisla(n):
    prvocisla = [2]
    i = prvocisla[-1]
    while len(prvocisla) < n:
        i += 1
        for prvocislo in prvocisla:
            if i % prvocislo == 0:
                break
        else:
            prvocisla.append(i)
    return prvocisla

In [ ]:
%%timeit
najdi_prvocisla(100)
najdi_prvocisla(10)

In [ ]:
%%prun
najdi_prvocisla(1000)


In [ ]:
print(najdi_prvocisla(100))

In [ ]:
%cd ..
%rm -rf test
%pwd

In [ ]:
%ll

# Třídy (class)
Třídou je jakýkoliv uživatelský typ. Podobně jako vestavěné typy nabízí metody a data (atributy), ovšem můžeme je libovolně definovat.

Třída je definována pomocí klíčového slova `class` a následuje jej její název. Vnitřek třídy je odsazený o jednu úroveň. 


In [ ]:
class MojeTrida:
    pass

## Metody a atributy

Metoda je puze jiný název pro funkci, která je definována uvnitř třídy.
Definice metody musí být uvnitř bloku třídy. (*Pozn. Metody lze do třídy přidat i později, ale není to preferovaný způsob.*)

Atribut je proměnná, která je definována uvnitř třídy. Atributy mohou být jakékoliv typy, včetně funkcí (ale tomu bychom už zase říkaly metoda :-) pro Python je ale všechno objekt).

Běžné metody (**instance method**) se volají na konkrétním objektu. Kromě nich existují i tzv. **metody třídy** a **statické metody**, které zde nebudeme probírat.

Zvláštnost (*Pozn. Ano, je to opravdu divné.*) definice metod (narozdíl od C++, Javy a dalších jazyků) je ta, že první argument metody je objekt, na kterém je metoda volána. Bez toho by metoda vůbec nevěděla, se kterým objektem pracuje! Dle konvence (která se snad nikdy neporušuje) se tento argument nazývá **self**. Při volání metody se pak vynechává a Python jej automaticky doplní.

Nastavení atributu a jeho hodnoty se provádí podobně jako ukládání do proměnné, ale musíme přidat objekt a tečku. (*Pozn. Interně jsou atributy uložené ve slovnících a při přístupu k nim se prochází slovník samotného objektu, jeho třídy, jejích nadřazených tříd, ...*). Atribut daného jména nemusí přitom vůbec existovat, nemusí se nijak deklarovat.

## Konstruktor
Metoda, která inicializuje objekt - zavolá se na prázdném objektu ve chvíli, kdy vytvoříme novou instanci.
Můžeme jej definovat, ale nemusíme - v takovém případě se použije výchozí konstruktor, který jednoduše nedělá nic (zvláštního). Konstruktor se v Pythonu vždy jmenuje **\_\_init\_\_** (dvě podtržítka před i po).

In [ ]:
# ukázka jednoduché třídy MojeTrida
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr # atribut

    def vypis_muj_parametr(self): # metoda
        print("Atribut ma hodnotu: ", self.muj_parametr)
        
objekt_tridy = MojeTrida("Ahoj") # vytvoření objektu pomocí konstruktoru

objekt_tridy.vypis_muj_parametr() # volání metody

print(objekt_tridy.muj_parametr) # přístup k atributu

V jiných jazicích se můžeme setkat s privátními/chraněnými atributy a metodami, které jsou přístupné jen zvnitř třídy. V Pythonu je to trochu jinak. Všechny atributy a metody jsou veřejné, ale jejich názvy jsou konvencí označeny jako privátní. **Všechny, které začínají podtržítkem `_`, jsou privátní** a neměly by být přístupné zvenčí třídy. V Pythonu je to ale jen konvence, která není vůbec kontrolována a můžeme se k něčemu takovému dostat i zvenčí třídy.

## Property (vlastnosti)
Vlastnosti jsou "chytřejší" data. Umožnují vstoupit do procesu čtení nebo nastavování atributu. Hodí se to například tehdy, pokud objekt má několik navzájem závislých parametrů a my je nechceme ukládat nezávisle; pokud chceme kontrolovat, jaká hodnota se ukládá; či pokud chceme s ukládanou nebo čtenou hodnotou ještě něco zajímavého provést (viz příklad pro kruh).

Ze syntaktického hlediska musíme nejdříve definovat metodu, která nese jméno vlastnosti a která tuto vlastnost "čte" (resp. vrací její hodnotu). O řádek výše musíme umístit tzv. *dekorátor* (tento koncept teď nebudeme podrobně vysvětlovat, jen jej pasivně použijeme) **@property**. Chceme-li, můžeme pak vytvořit i metodu pro zápis - ta se musí jmenovat stejně, požadovat jeden argument (ukládaná hodnota) a být uvedena dekorátorem **@*jmenovlastnosti*.setter**. Podobně bychom mohli vytvořit i metodu pro mazání (dekorátor **@*jmenovlastnosti*.deleter**), ale to se již běžně nedělá.

Jakmile máme takto vytvořené vlastnosti, přistupujeme k nim jako k běžným datovým atributům - voláme je bez závorek a přiřazujeme do nich pomocí znaménka "rovná se".

*Pozn. Vlastnosti fungují podobně jako properties v C# či javabeans v Javě. Povšimněte si však, že pro přístup k vlastnostem se používá úplně stejný zápis jako pro přístup k datovým atributům. Pokud tedy budeme chtít někdy změnit chování datového atributu a udělat z něj vlastnost, klient naší třídy to nepozná a nebude muset dělat žádné změny v kódu. Není tedy vhodné přespříliš iniciativně vytvářet triviální vlastnosti, které jen obalují přístup k atributům (jako by se to jistě dělalo v Javě).*

In [ ]:
import math

class Kruh:
    def __init__(self, r):
        self.polomer = r

    @property                           # Chceme definovat vlastnost pro čtení
    def obsah(self):                    # Vypadá jako obyčejná metoda
        return math.pi * self.polomer ** 2

    @obsah.setter                       # Chceme nastavit zápis do dříve definované vlastnosti
    def obsah(self, s):
        print("Měním obsah na {}".format(s))
        self.polomer = math.sqrt(s / math.pi)

    @obsah.deleter
    def obsah(self):
        pass


kruh = Kruh(1)
print(kruh.polomer)    # Normální datový atribut
print(kruh.obsah)      # Property

kruh.obsah = 3                          # Změníme obsah pomocí zapisovatelné vlastnosti
print(kruh.polomer)    # Normální datový atribut
print(kruh.obsah)      # Property


## Podtržítkové konvence
V Pythonu obecně jsou konvence velice silně zakořeněné. Na objektech to je vidět snad nejvíce.

- "Soukromé" atributy (atributem se v Pythonu často rozumí jak data tak metody -- vše je objekt) se pojmenovávají s podtržítkem na začátku, tj. např. `_soukroma_metoda`.
- Atributy se dvěma podtžítky na začátku i na konci mají speciální význam (viz [dokumentace](http://docs.python.org/3/reference/datamodel.html#special-method-names)). Už jsme viděli `__init__`, podíváme se na několik dalších.
    * `__repr__` a `__str__` převádějí objekt na řetězec.
    * `__getattr__` a `__setattr__` slouží pro čtení a ukládání nenalezených atributů.
    * `__call__` se zavolá pokud použijeme objekt jako funkci.
    * `__doc__` obsahuje dokumentaci (docstring).
    * `__dict__` obsahuje slovník se jmenným prostorem objektu.
    * ... dále existují speciální funkce pro logické operátory, pro emulaci funkcionality kontejnerů (iterace, položky, řezy), pro aritmetické operace atd.

In [ ]:
# ukázka __getattr__ a __setattr__
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def __getattr__(self, item):
        print("Zadáváte neexistující atribut: ", item)
        return "Ahoj"

    def __setattr__(self, key, value):
        print("Nastavuji hodnotu atributu: ", key, " na hodnotu: ", value)
        object.__setattr__(self, key, value)
        
        
objekt_tridy = MojeTrida("Ahoj")
print(dir(objekt_tridy))
print(objekt_tridy.tento_parametr_neexistuje)
objekt_tridy.definuji_novy_atribut = "Nazdarek"
print(objekt_tridy.definuji_novy_atribut)
print(dir(objekt_tridy))


In [ ]:
# ukázka __str__ a __repr__
# rozdíl mezi __str__ a __repr__ je v tom, že __str__ je volána při převodu objektu na řetězec 
# a __repr__ je volána při výpisu objektu (např. v interaktivním režimu)
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def __str__(self):
        return "Toto je objekt tridy MojeTrida"

    def __repr__(self):
        return "MojeTrida s parametrem: " + str(self.muj_parametr)
    
objekt_tridy = MojeTrida("Ahoj")
print(objekt_tridy)
print(str(objekt_tridy))
print(repr(objekt_tridy))
objekt_tridy

In [ ]:
# seznam atributů __dict__
class MojeTrida:
    def __init__(self, muj_parametr1, muj_parametr2):
        self.muj_parametr1 = muj_parametr1
        self.muj_parametr2 = muj_parametr2

    def vypis_moje_parametry(self):
        print("Atributy mají hodnotu: ", self.muj_parametr, ", ", self.muj_parametr2)

objekt_tridy = MojeTrida("Ahoj", "Sbohem")
print(objekt_tridy.__dict__)

In [ ]:
# ukázka __call__
class MojeTrida:
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def __call__(self, *args, **kwargs):
        print("Voláte objekt tridy MojeTrida")
        
objekt_tridy = MojeTrida("Ahoj")
objekt_tridy()

In [ ]:
# ukázka __doc__
class MojeTrida:
    """Toto je ukázka dokumentace třídy MojeTrida"""
    def __init__(self, muj_parametr):
        self.muj_parametr = muj_parametr

    def vypis_muj_parametr(self):
        """Toto je ukázka dokumentace metody vypis_muj_parametr"""
        print("Atribut ma hodnotu: ", self.muj_parametr)
        
objekt_tridy = MojeTrida("Ahoj")
print(objekt_tridy.__doc__)
print(objekt_tridy.vypis_muj_parametr.__doc__)

### Podtržítkové metody implementující algebraické operace
V Pythonu je možné nadefinovat chování operátorů (+,-,*,/,...) pro vlastní objekty. Toho můžeme docílit tak, že nadefinujeme speciální metody:
- `__add__` pro operátor `+`
- `__sub__` pro operátor `-`
- `__mul__` pro operátor `*`
- `__div__` pro operátor `/`
- `__truediv__` pro operátor `/` (v Pythonu 3 je toto stejné jako `__div__`)
- `__floordiv__` pro operátor `//`
- `__mod__` pro operátor `%`
- `__pow__` pro operátor `**`
- `__lshift__` pro operátor `<<`
- `__rshift__` pro operátor `>>`
- `__and__` pro operátor `&`
- `__xor__` pro operátor `^`
- `__or__` pro operátor `|`
- a další (viz [dokumentace](http://docs.python.org/3/reference/datamodel.html#special-method-names)).

In [ ]:
# ukázka třídy implementující algebraické operátory (zlomek)
class Zlomek:
    def __init__(self, citatel, jmenovatel):
        self.citatel = citatel
        self.jmenovatel = jmenovatel

    def __add__(self, other):
        return Zlomek(self.citatel * other.jmenovatel + self.jmenovatel * other.citatel, self.jmenovatel * other.jmenovatel)

    def __sub__(self, other):
        return Zlomek(self.citatel * other.jmenovatel - self.jmenovatel * other.citatel, self.jmenovatel * other.jmenovatel)

    def __mul__(self, other):
        return Zlomek(self.citatel * other.citatel, self.jmenovatel * other.jmenovatel)

    def __truediv__(self, other):
        return Zlomek(self.citatel * other.jmenovatel, self.jmenovatel * other.citatel)

    def __str__(self):
        return str(self.citatel) + " / " + str(self.jmenovatel)

    def __repr__(self):
        return "Zlomek(" + str(self.citatel) + ", " + str(self.jmenovatel) + ")"
    
zlomek1 = Zlomek(1, 2)
zlomek2 = Zlomek(1, 3)
print(zlomek1 + zlomek2)
print(zlomek1 - zlomek2)
print(zlomek1 * zlomek2)
print(zlomek1 / zlomek2)

Třídy mohou také definovat funkce `__getitem__` a `__setitem__` díky kterým je možné přistupovat k objektu jako k poli/slovníku. Tato chování se pak použije v případě, že se použije operátor `[]` (např. `obj[1]`).

In [ ]:
# ukázka __setitem__ a __getitem__
class MojeTrida:
    def __init__(self, muj_parametr):
        self._moje_data = dict()
        self.muj_parametr = muj_parametr + " "

    def __setitem__(self, key, value):
        print("Nastavuji hodnotu atributu: ", key, " na hodnotu: ", value)
        self._moje_data[key] = self.muj_parametr + str(value)

    def __getitem__(self, item):
        return self._moje_data[item]
    
objekt_tridy = MojeTrida("Ahoj")
objekt_tridy[0] = "Honzo"
objekt_tridy[1] = "Pepo"
objekt_tridy[2] = "Jardo"

print(objekt_tridy[0])
print(objekt_tridy[1])
print(objekt_tridy[2])


## Dědičnost
Třída může svoje chování (i data) odvozovat od nějaké jiné třídy, čímž si ušetříme spoustu práce při opakování společných rysů. V takovém případě řekneme, že naše nová třída (dceřinná) od té původní (rodičovské) dědí.

* V dceřinné třídě můžeme změnit definici některé metody z rodičovské třídy.
* Konstruktory se standardně dědí (*Na rozdíl od C++ či Javy, kde se musí explicitně volat, v Pythonu se volají jen pokud definujeme nový konstruktor a chceme zavolat i nadřazený.*)
* Instance dceřinné třídy se mohou použít kdekoliv, kde počítá s objektem rodičovské třídy. *Toto platí v Pythonu ještě obecněji - obvykle se nekontrolují konkrétní typy, projde jakýkoliv objekt, který nabízí používané atributy/metody.*

**Syntax:** Jméno rodičovské třídy se dává do závorky za jméno (místo object, od kterého třídy obvykle dědí).

In [ ]:
class Clovek:
    def __init__(self, jmeno):         # Konstruktor, který nastaví atribut "jmeno"
        self.jmeno = jmeno
        
    def _rekni(self, text):            # Privátní metoda, která vypíše text
        print(self.jmeno, ": ", text)

    def predstav_se(self):
        self._rekni("Jmenuji se " + self.jmeno + ".")

    def pozdrav(self):
        self._rekni("Dobrý den.")

    def rozluc_se(self):
        self._rekni("Nashledanou.")


class Elektrikar(Clovek):
    def oprav_televizi(self):         # Nová metoda v rodičovské třídě - jiný Clovek ji neumí
        self._rekni("Bude to v cuku letu.")
        print("---Elektrikar něco šudlá.---")
        self._rekni("A je to.")

    def predstav_se(self):            # Předefinovaná metoda "predstav_se" využívá atribut rodičovské třídy
        self._rekni("Já sem ňákej " + self.jmeno + ".")


class Zakaznik(Clovek):
    def nakupuj(self):                # Nová metoda v rodičovské třídě - jiný Clovek ji neumí
        self._rekni("Prosím opravíte mi, televizi.")


e = Elektrikar("Franta Vopička")
z = Zakaznik("Tomáš Marný")

# Rozhovor
z.pozdrav()          # Všimněte si, že "pozdrav" je rodičovská metoda, ale volá se "rekni" z dceřinné třídy.
e.pozdrav()
z.predstav_se()
e.predstav_se()
z.nakupuj()
e.oprav_televizi()

## Dataclass
Od Pythonu 3.7 je možné používat dekorátor `@dataclass`, který umožňuje jednoduše definovat třídy, které se pouze skládají z datových atributů.

Výhody použití `@dataclass` je automatické generování několika metod:
- `__init__` (konstruktor)
- `__repr__` (reprezentace objektu jako řetězce)
- `__eq__` (rovnost)
- `__lt__` (menší než)
- `__le__` (menší nebo rovno)
- `__gt__` (větší než)
- `__ge__` (větší nebo rovno)
- ... a další (viz [dokumentace](https://docs.python.org/3/library/dataclasses.html#dataclasses.dataclass)).

In [ ]:
from dataclasses import dataclass

@dataclass(order=True)
class Point3D:
    x: float
    y: float
    z: float
    
bod1 = Point3D(1, 2, 3)
bod2 = Point3D(1, 3, 2)

print(bod1)
print(bod1 < bod2) # jak je toto definováno? viz dokumentace


## Další témata (zde se jimi nebudeme zabývat)
Ale jsou to věci, které je užitečné znát, alespoň vědět, že existují.

* Vícenásobná dědičnost
* Metody třídy
* Statické metody
* Abstraktní třídy
* Metatřídy (Dataclass, ...)
* Návrhové vzory


# Výjimky a chyby
Chyby v Pythonu můžeme rozlišit na *syntaktické chyby* a *chyby za běhu* (*run-time*). Syntaktické chyby jsou způsobeny např. nespárovanými závorkami, špatným odsazením apod. Program se syntaktickými chybami nelze vůbec spustit. Častější jsou run-time chyby, které vznikají nesprávným použitím nějaké funkce, chybějícími daty apod. Jelikož je Python interpretovaný jazyk, většina chyb se ukáže až za běhu, na rozdíl od kompilovaných jazyků, kde se mnoho chyb objeví při kompilaci.
<!-- TEASER_END -->

## Výjimky

Výjimka (*exception*) je vyhozena ve chvíli, kdy dojde k chybě (respektivě ve chvíli kdy autor funkce kterou používáme považuje naše užití za chybné :-)). Pokud tuto výjimku nezachytíme (viz dále), běh programu se přeruší. Např. dělení nulou skončí výjimkou `ZeroDivisionError`:

### Chytáme výjimky
Pokud nechceme, aby běh programu skončil ve chvíli výjimky, můžeme použít `try` - `except` blok. Ten funguje tak, že rizikovou část kód umístíme do `try` bloku, do `except` bloku pak umístíme instrukce pro případ chyby (výjimky).

In [ ]:
# ukázka try/except

def deleni(a, b):
    return a / b

def deleni_upravene(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return "Nelze dělit nulou!"
    
print(deleni_upravene(1, 0))
print(deleni_upravene(1, 2))
print(deleni(1, 0))

`except` lze také použít bez specifikace typu výjimky, v tom případě se zachytí všechny výjimky. Toto ale není dobrá praxe, protože vlastně nevíme jaký typ chyby "obcházíme".

Kompletní try-except blok může ještě obsahovat `else` a `finally` bloky, viz [dokumentace](http://docs.python.org/3/reference/compound_stmts.html#try). `finally` se hodí zejména pro "úklid", např. zavření souboru apod.

In [ ]:
# kopletní try/except/else/finally
a = 1
b = 0
try:
    c = a / b
except:
    print("Nastala nějaká chyba, nestarám se o to jaká.")
else:
    print("Všechno v pořádku.")
finally:
    print("Toto se vždy provede.")

Pomocí modulu traceback si můžeme nechat vypsat podrobnější informace jak k vyjímce došlo, to se může hodit při ladění.

In [ ]:
import traceback

a = 1
b = 0
try:
    c = a / b
except:
    print("Nastala nějaká chyba, nestarám se o to jaká.")
    traceback.print_exc() # dá se nastavit na výstup do souboru
else:
    print("Všechno v pořádku.")
finally:
    print("Toto se vždy provede.")

print("A stale jedeme dál.")


### Vytváříme vlastní výjimky
Výjimku můžeme samozřejmě vyhodit i v našem kódu pomocí klíčového slova `raise`. Pokud bychom chtěli např. kontrolovat vstup nějaké funkce, uděláme to takto:

In [ ]:
# ukázka raise
def zaplatit(cena):
    if cena > 100:
        raise ValueError("Cena je moc vysoká!")
    else:
        print("Zaplatil jsem", cena, "Kč.")
        
zaplatit(50)
zaplatit(150)

# Práce s řetězci
S řetězci jsme se už setkali, řekli jsme si také, že `str` objekt je immutable (neměnitelný).

## Základní operace
* `+` - spojení dvou řetězců
* `*` - opakování řetězce
* `in` - zjištění, zda je řetězec podřetězcem jiného řetězce
* `[]` - indexování (přístup k jednotlivým znakům)
* `[:]` - slicing (přístup k podřetězcům)
* `len()` - délka řetězce
* `str()` - převod na řetězec


In [ ]:
# ukázky základních operací s řetězci
retezec = "ahoj"
retezec2 = "svete"
retezec3 = "tak tedy ahoj zeměkoule" 

print(retezec + retezec2)
print(retezec * 3)
print(retezec[0:2])
print(len(retezec))
print(retezec in retezec3)
print(retezec2 in retezec3)

## Vestavěné metody řetězců
Velká/malá písmena:
* `str.capitalize()` - první znak velký, ostatní malé
* `str.swapcase()` - velká písmena na malá, malá na velká
* `str.title()` - první znak každého slova velký, ostatní malé
* `str.lower()` - malá písmena
* `str.upper()` - velká písmena

Ořezávání:
* `str.strip([chars])` - ořezání bílých znaků (mezera, tabulátor, nový řádek) na začátku a na konci
* `str.lstrip([chars])` - ořezání bílých znaků na začátku
* `str.rstrip([chars])` - ořezání bílých znaků na konci
* `str.center(width[, fillchar])` - vycentrování řetězce
* `str.ljust(width[, fillchar])` - zarovnání vlevo
* `str.rjust(width[, fillchar])` - zarovnání vpravo

Nahrazování/řezání/spojování řetězců:
* `str.format(*args, **kwargs)` - umožňuje definovat placeholdery v řetězci `{}` a ty pak naplnit hodnotami
* `str.replace(old, new[, count])` - nahrazení všech výskytů podřetězce
* `str.split([sep[, maxsplit]])` - rozdělení řetězce na seznam řetězců
* `str.splitlines([keepends])` - rozdělení řetězce na řádky
* `str.partition(sep)` - rozdělení řetězce na 3 části (před, sep, za)
* `str.join(iterable)` - spojení řetězců v seznamu
* `str.expandtabs(tabsize=8)` - nahrazení tabulátorů mezerami

Počítání výskytů/znaků/...:
* `str.count(sub[, start[, end]])` - počet výskytů podřetězce
* `str.find(sub[, start[, end]])` - index prvního výskytu podřetězce nebo -1 pokud není nalezen
* `str.index(sub[, start[, end]])` - index prvního výskytu podřetězce nebo vyhodí výjimku `ValueError` pokud není nalezen

Zjištění některých vlastností řetězců:
* `str.startswith(prefix[, start[, end]])` - zjištění, zda řetězec začíná daným podřetězcem
* `str.endswith(suffix[, start[, end]])` - zjištění, zda řetězec končí daným podřetězcem
* `str.isalnum()` - zjištění, zda řetězec obsahuje pouze alfanumerické znaky
* `str.isalpha()` - zjištění, zda řetězec obsahuje pouze písmena
* `str.isdecimal()` - zjištění, zda řetězec obsahuje pouze desetinná čísla
* `str.isdigit()` - zjištění, zda řetězec obsahuje pouze čísla
* `str.isidentifier()` - zjištění, zda řetězec je platným identifikátorem
* `str.islower()` - zjištění, zda řetězec obsahuje pouze malá písmena
* `str.isnumeric()` - zjištění, zda řetězec obsahuje pouze čísla
* `str.isprintable()` - zjištění, zda řetězec je tisknutelný
* `str.isspace()` - zjištění, zda řetězec obsahuje pouze bílé znaky
* `str.istitle()` - zjištění, zda řetězec obsahuje pouze velká písmena


In [ ]:
retezec = "ahoj Svete toto je retezec"
print(retezec.capitalize())
print(retezec.swapcase())
print(retezec.title())
print(retezec.lower())
print(retezec.upper())

In [ ]:
retezec = "  ahoj Svete toto je retezec      "
print(retezec.strip())
print(retezec.lstrip())
print(retezec.rstrip())
print(retezec.center(50, "-"))
print(retezec.ljust(50, "."))
print(retezec.rjust(50, " "))

In [ ]:
# Základní formátování
jmeno = "Jan"
print("Ahoj, {}!".format(jmeno))
# Výstup: Ahoj, Jan!

# Formátování s formátovacím specifikátorem
vek = 30
print("{} je {} let starý.".format(jmeno, vek))
# Výstup: Jan je 30 let starý.

# Formátování s pozicemi argumentů
print("{0} je {1} let starý. Nejlepším přítelem {0} je {2}.".format(jmeno, vek, "Jana"))
# Výstup: Jan je 30 let starý. Nejlepším přítelem Jan je Jana.

# Formátování s klíčovými argumenty
print("{jmeno} je {vek} let starý. Nejlepším přítelem {jmeno} je {pritel}.".format(
    jmeno=jmeno, vek=vek, pritel="Jana"))
# Výstup: Jan je 30 let starý. Nejlepším přítelem Jan je Jana.

In [ ]:
text = "Ahoj světe, \njak se máš?"
print(text)

print(text.replace("Ahoj", "Nazdar"))
print(text.split())
print(text.splitlines())
print(text.partition(", "))

In [ ]:
seznam = ['Ahoj', 'světe']
print(', '.join(seznam))

text = "Ahoj\tsvěte"
print(text.expandtabs())

In [ ]:
text = "Ahoj světe, jak se máš?"
print(text.count("e"))
print(text.find("světe"))
print(text.index("světe"))


In [ ]:
text = "Ahoj světe, jak se máš?"
print(text.startswith("Ahoj"))
print(text.startswith("Nazdar"))
print(text.endswith("?"))
print(text.endswith("!"))

## f-strings
F-stringy (formatovací řetězce) jsou speciální syntaxe, která umožňuje vložit proměnné do řetězce pomocí zápisu {} a začínají znakem "f".

In [ ]:
# f-strings
jmeno = "Jan"
vek = 30
print(f"{jmeno} je {vek} let starý.")

f-strings umožňují formátovat hodnoty pomocí specifikátorů, podrobnosti viz [dokumentace](https://docs.python.org/3/library/string.html#format-specification-mini-language).

In [ ]:
# f-strings s formátovacím specifikátorem
jmeno = "Jan"
vek = 30
print(f"{jmeno} je {vek:.2f} let starý.")
print(f"{jmeno:<10} má {vek + 3:>3} let")
print(f"{jmeno =}, {vek =}")


In [ ]:
# formátování pomocí > a < můžeme využít pro zarovnání tabulky
print(f"{'číslo':>6} {'mocnina':>8} {'třetí mocnina':>14}")
for i in range(1, 20):
    print(f"{i:.>6} {i**2:.>8} {i**3:.>14}")

Možnosti formátování f-stringu jsou vestavěné do třídy kterou vypisujeme, konkrétně metoda `__format__`. Tato metoda je volána při použití f-stringu a použití `:`.

In [ ]:
class Bod3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __format__(self, format_spec):
        if format_spec == 'zyx':
            return f"{self.z}, {self.y}, {self.x}"
        elif format_spec == 'yzx':
            return f"{self.y}, {self.z}, {self.x}"
        else:
            return f"{self.x}, {self.y}, {self.z}"
        
bod = Bod3D(1, 2, 3)
print(f"{bod = }")
print(f"{bod:zyx}")
print(f"{bod:yzx}")
print(f"{bod = :}")


# Generátory a iterátory
Abychom se v tomto tématu lépe zorientovali, začneme s rozčleněním. Budeme používat tyto tři termíny:
- `iterable` (iterovatelný objekt) - objekt, který umí vracet své prvky jeden po druhém
    - kontejnery (list, slovník, ...)
    - řetězec
    - range
    - objekt typu stream (např. `file`)
- `iterator` (iterátor) - objekt, který umí iterovat (implementuje tzv. protokol iterátoru). Lze vytvořít z iterovatelného objektu pomocí funkce `iter()`.
    - metoda `__iter__` vrací samotný objekt
    - metoda `__next__` vrací další prvek (na konci vyhodí výjimku `StopIteration`)
- `generator` (generátor) - je typ iterátoru. Má dvě varianty:
    - generátor funkce - funkce, která vrací výstupy postupně pomocí `yield` místo `return`
    - generátor výraz - výraz využívající syntaxe: **(**`výraz` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**)**


  Pěkné vysvětlení lze najít i zde: [Iterables vs. Iterators vs. Generators](http://nvie.com/posts/iterators-vs-generators/)

In [ ]:
# ukázka iterable objektů
muj_list = [1, 2, 3, 4, 5]
muj_string = "ahoj"
muj_double = 1.5
print(iter(muj_list))
print(iter(muj_string))
print(iter(muj_double))

In [ ]:
# ukázka iterátorů
muj_list = [1, 2, 3, 4, 5]
muj_iterator = iter(muj_list)
print(muj_iterator.__iter__())
print(muj_iterator.__next__())
print(next(muj_iterator))
print(next(muj_iterator))
print(muj_iterator.__next__())
print(next(muj_iterator))
print(next(muj_iterator))

In [ ]:
# generator funkce
def generator1():
    yield "Ahoj"
    yield "světe"
    yield "jak"
    yield "se"
    yield "máš?"


def generator2(start, konec):
    for i in range(start, konec):
        print("A tohle je další iterace!")
        yield i

# ukázka použití generátoru
gen1 = generator1()
print(gen1)
print(next(gen1))
print(next(gen1))
print(next(gen1))
print(next(gen1))
print(next(gen1))
try:
    print(next(gen1))
except StopIteration:
    print(f"Generátor {gen1} vyčerpal všechny hodnoty.")
    
gen2 = generator2(1, 6)
print(gen2)
print(next(gen2))
print(next(gen2))
print(next(gen2))
print(next(gen2))
print(next(gen2))
try:
    print(next(gen2))
except StopIteration:
    print(f"Generátor {gen2} vyčerpal všechny hodnoty.")


In [ ]:
# generatorový výraz
gen3 = (i for i in range(1, 6))
print(gen3)
print(next(gen3))
print(next(gen3))
print(next(gen3))
print(next(gen3))
print(next(gen3))
try:
    print(next(gen3))
except StopIteration:
    print(f"Generátor {gen3} vyčerpal všechny hodnoty.")

In [ ]:
# generatorový výraz s podmínkou
gen4 = (i for i in range(1, 8) if i % 2 == 0)
print(gen4)
print(next(gen4))
print(next(gen4))
print(next(gen4))
try:
    print(next(gen4))
except StopIteration:
    print(f"Generátor {gen4} vyčerpal všechny hodnoty.")

# List/Set/Dict comprehensions
List/Set/Dict comprehensions jsou způsob, jak vytvořit seznam, množinu nebo slovník pomocí jednoho řádku kódu. Všechny tyto konstrukce jsou velmi podobné jako generator výrazy.
- List comprehension - vytvoří seznam, syntaxe: **[**`výraz` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**]**
- Set comprehension - vytvoří množinu, syntaxe: **{**`výraz` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**}**
- Dict comprehension - vytvoří slovník, syntaxe: **{**`klíč` **:** `hodnota` **for** `proměnná` **in** `iterovatelný_objekt` **if** `podmínka`**}**

In [ ]:
# list comprehension
print([i for i in range(1, 6)])
print([i for i in range(1, 6) if i % 2 == 0])
print([i + 1 for i in range(1, 6) if i % 2 == 0])
print([(i, i**2, i**3) for i in range(1, 6)])

In [ ]:
# set comprehension
print({i for i in range(1, 6)})
print({i for i in range(1, 6) if i % 2 == 0})
print({i + 1 for i in range(1, 6) if i % 2 == 0})
print({(i, i**2, i**3) for i in range(1, 6)})

In [ ]:
# dict comprehension
print({i: i**2 for i in range(1, 6)})
print({i: i**2 for i in range(1, 6) if i % 2 == 0})
print({i + 1: i**2 for i in range(1, 6) if i % 2 == 0})
print({i: (i**2, i**3) for i in range(1, 6)})

# Základní práce se soubory
Pro práci se sounory je základní příkaz `open`, který vrací instanci `file`

In [ ]:
%%writefile test.txt
Ahoj, jak se máš?
To se mi líbí!
To ne.
To ano.
To ne.
To ano.

In [ ]:
# ukázka otevření souboru
soubor = open("test.txt", "r")
print(soubor)

In [ ]:
# metody file
print([m for m in dir(soubor) if not m.startswith("_")])

In [ ]:
print(soubor.read())
soubor.close()

Soubor se dá procházet jakožto iterovatelný objekt, který vrací řádky souboru. Pozor na "\n" na konci řádku.

In [ ]:
# procházení řádků souboru
soubor = open("test.txt", "r")
for idx_radku, radek in enumerate(soubor):
    print(f"{idx_radku = }, {radek = }")
soubor.close()

Pro práci se soubory je velice užitečný (*a jednoznačně preferovaný*) blok [`with`](https://docs.python.org/3/reference/compound_stmts.html#with), který používá tzv. context managery. To nám zajistí, že soubor bude vždy zavřen, i když během práce se souborem dojde k chybě (neodchycená výjimka). Nahrazuje to tak `try..except..finally` blok. 

Použití je velice jednoduché, předchozí příklad by vypadal takto:

In [ ]:
# ukázka užití with
with open("test.txt", "r") as soubor:
    for idx_radku, radek in enumerate(soubor):
        print(f"{idx_radku = }, {radek = }")

Zapisování do souboru je velice podobné, pouze je potřeba přidat parametr `w` (write) nebo `a` (append).

In [ ]:
# vytvoření csv souboru s čísly a jejich mocninami
with open("mocniny.csv", "w", newline="") as soubor:
    soubor.write("čislo, mocnina \n")
    for i in range(1, 10):
        soubor.write(f"{i}, {i*i} \n")

In [ ]:
%less mocniny.csv
